<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Migrate-AQ-Food-api-entities" data-toc-modified-id="Migrate-AQ-Food-api-entities-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Migrate AQ Food api entities</a></span></li></ul></div>

Migrate AQ Food api entities:  
    * Datasets with layers and widgets associated plus metadata amd vocabulary.  
    * Update the notebooks for layers and widgets to reflect the new layer/widget and dataset id  
Migrate AQ Risk api entities:  
    * Datasets with layers plus metadata  
Migrate AQ Rankings api entities:  
    * Datasets with layers plus metadata  

Set up the micro in production For floods and Risk  


# Migrate AQ Food and AQ country ranking api entities

In [1]:
import getpass
import requests as re
import json
from datetime import datetime

In [2]:
staging_server = "https://staging-api.globalforestwatch.org"
prod_server = "https://api.resourcewatch.org"

In [3]:
def auth():
    with re.Session() as s:
        headers = {'Content-Type': 'application/json'}
        payload = json.dumps({ 'email': f'{input("Email: ")}',
                               'password': f'{getpass.getpass(prompt="Password: ")}'})
        response = s.post(f'{prod_server}/auth/login',  headers = headers,  data = payload)
        response.raise_for_status()
    return response.json().get('data').get('token')

token = auth()

Email: alicia.arenzana@vizzuality.com
Password: ········


In [4]:
def getUrl(url, payload):
    response = re.get(url, payload)
    response.raise_for_status()
    return response.json()

def DeleteAssets(url):
    response = re.delete(url, headers = {'Authorization':f'Bearer {token}'})
    response.raise_for_status()
    return response.status_code

def CreateAssets(url, body):
    response = re.post(url, data=json.dumps(body), headers = {'Authorization':f'Bearer {token}', 'Content-Type': 'application/json'})
    if response.status_code !=200:
        print(response.text)
        print(url)
        print(json.dumps(body))
    response.raise_for_status()
    return response.json()

def recreateDataset(dataset):
    
    if dataset.get('type')!='dataset':
        return None
    
    url = f'{prod_server}/v1/dataset'
    body = {'dataset':{
        'application': dataset['attributes'].get('application'),
        'name': dataset['attributes'].get('name'),
        'connectorType': dataset['attributes'].get('connectorType'),
        'provider': dataset['attributes'].get('provider'),
        'published': dataset['attributes'].get('published'),
        'overwrite': dataset['attributes'].get('overwrite'),
        'mainDateField': dataset['attributes'].get('mainDateField'),
        'env': dataset['attributes'].get('env'),
        'geoInfo': dataset['attributes'].get('geoInfo'),
        'protected': dataset['attributes'].get('protected'),
        'legend': dataset['attributes'].get('legend'),
        'widgetRelevantProps': dataset['attributes'].get('widgetRelevantProps'),
        'layerRelevantProps': dataset['attributes'].get('layerRelevantProps')
        }
    }
    if dataset['attributes'].get('provider') == 'cartodb':
            body['dataset']['connectorUrl'] =  dataset['attributes'].get('connectorUrl')
    if dataset['attributes'].get('provider') == 'gee':
            body['dataset']['tableName'] =  dataset['attributes'].get('tableName')
    response = CreateAssets(url, body)
    return response

def recreateLayer(datasetId, layer):
    if layer.get('type')!='layer':
        return None
    
    url = f'{prod_server}/v1/dataset/{datasetId}/layer'

    body = {
        'application': layer['attributes'].get('application'),
        'name': layer['attributes'].get('name'),
        'iso': layer['attributes'].get('iso'),
        'provider': layer['attributes'].get('provider'),
        'default': layer['attributes'].get('default'),
        'protected': layer['attributes'].get('protected'),
        'published': layer['attributes'].get('published'),
        'env': layer['attributes'].get('env'),
        'layerConfig': layer['attributes'].get('layerConfig'),
        'legendConfig': layer['attributes'].get('legendConfig'),
        'interactionConfig': layer['attributes'].get('interactionConfig'),
        'applicationConfig': layer['attributes'].get('applicationConfig'),
        'staticImageConfig': layer['attributes'].get('staticImageConfig')
    }
    
    response = CreateAssets(url, body)
    return response

def recreateWidget(datasetId, widget):
    if widget.get('type')!='widget':
        return None
    
    url = f'{prod_server}/v1/dataset/{datasetId}/widget'
    body = {
        'application': widget['attributes'].get('application'),
        'name': widget['attributes'].get('name'),
        'description': widget['attributes'].get('description'),
        'verified': widget['attributes'].get('verified'),
        'default': widget['attributes'].get('default'),
        'protected': widget['attributes'].get('protected'),
        'defaultEditableWidget': widget['attributes'].get('defaultEditableWidget'),
        'published': widget['attributes'].get('published'),
        'freeze': widget['attributes'].get('freeze'),
        'env': widget['attributes'].get('env'),
        'queryUrl': widget['attributes'].get('queryUrl'),
        'widgetConfig': widget['attributes'].get('widgetConfig'),
        'template': widget['attributes'].get('template'),
        'layerId': widget['attributes'].get('layerId')
    }
    response = CreateAssets(url, body)
    return response

def recreateMetadata(datasetId, metadata, layerId=None, widgetId=None):
    if metadata.get('type')!='metadata':
        return None
    if layerId and widgetId:
        raise Exception("layerId and widgetId not allowed at the same time")
    elif layerId:
        url = f'{prod_server}/v1/dataset/{datasetId}/layer/{layerId}/metadata'
    elif widgetId:
        url = f'{prod_server}/v1/dataset/{datasetId}/widget/{widgetId}/metadata'
    else:
        url = f'{prod_server}/v1/dataset/{datasetId}/metadata'
    
    body = {
        'application': metadata['attributes'].get('application'),
        'language': metadata['attributes'].get('language'),
        'description': metadata['attributes'].get('description'),
        'info': metadata['attributes'].get('info'),
    }
    if metadata['attributes'].get('name'):
        body['name'] = metadata['attributes'].get('name')
    
    response = CreateAssets(url, body)
    return response

def recreateVocabulary(datasetId, vocabulary):
    if vocabulary.get('type')!='vocabulary':
        return None
    url = f"{prod_server}/v1/dataset/{datasetId}/vocabulary/{vocabulary['attributes']['name']}"
    body = {
        'application': vocabulary['attributes'].get('application'),
        'tags': vocabulary['attributes'].get('tags')
    }
    
    response = CreateAssets(url, body)
    return response

### Get all layers in staging

In [5]:
url = f'{staging_server}/v1/dataset'
payload={
    'application':'aqueduct',
    'status':'saved',
    'published':'true',
    'includes':'widget,layer,vocabulary,metadata',
    'page[size]':1613982331640
}

stagingData = getUrl(url,payload)
stagingData

{'data': [{'id': '05c330d7-49cf-42d6-bada-e255b8c8e115',
   'type': 'dataset',
   'attributes': {'name': 'Global growing crops under high water stress conditions',
    'slug': 'Global-growing-crops-under-high-water-stress-conditions-1490086842548',
    'type': None,
    'subtitle': None,
    'application': ['aqueduct'],
    'dataPath': None,
    'attributesPath': None,
    'connectorType': 'rest',
    'provider': 'cartodb',
    'userId': '58333dcfd9f39b189ca44c75',
    'connectorUrl': 'https://wri-01.carto.com/tables/aqueduct_projections_20150309/public',
    'sources': [],
    'tableName': 'aqueduct_projections_20150309',
    'status': 'saved',
    'published': True,
    'overwrite': False,
    'mainDateField': None,
    'env': 'production',
    'geoInfo': False,
    'protected': False,
    'legend': {'date': [],
     'region': [],
     'country': [],
     'nested': [],
     'integer': [],
     'short': [],
     'byte': [],
     'double': [],
     'float': [],
     'half_float': [],
 

### Clean layers in production

In [6]:
url = f'{prod_server}/v1/dataset'
payload = {
    'application':'aqueduct',
    'status':'saved',
    'published':'true',
    'includes':'widget,layer,vocabulary,metadata',
    'page[size]':1613982331640
}

prodData = getUrl(url,payload)

#save a backup of production data just in case we need to recreate it again

#with open(f'AQ_prod_backup_{datetime.now().strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
#    json.dump(prodData, outfile)

for dataset in prodData['data']:
    try:
        status = DeleteAssets(f"{prod_server}/v1/dataset/{dataset['id']}")
    except re.exceptions.HTTPError as err:
        pass
        

In [7]:
url = f'{prod_server}/v1/dataset'
payload = {
    'application':'aqueduct',
    'status':'saved',
    'published':'true',
    'includes':'widget,layer,vocabulary,metadata',
    'page[size]':1613982331640
}
getUrl(url,payload)

{'data': [{'id': '134caa0a-21f7-451d-a7fe-30db31a424aa',
   'type': 'dataset',
   'attributes': {'name': 'Political boundaries (GADM)',
    'slug': 'Country-geometries-1490086842548',
    'type': 'tabular',
    'subtitle': None,
    'application': ['gfw',
     'data4sdgs',
     'aqueduct',
     'forest-atlas',
     'gfw-climate'],
    'dataPath': None,
    'attributesPath': None,
    'connectorType': 'rest',
    'provider': 'cartodb',
    'userId': '57a0aa1071e394dd32ffe137',
    'connectorUrl': 'https://wri-01.carto.com/tables/gadm28_countries/public',
    'sources': [],
    'tableName': 'gadm28_countries',
    'status': 'saved',
    'published': True,
    'overwrite': False,
    'mainDateField': None,
    'env': 'production',
    'geoInfo': True,
    'protected': True,
    'legend': {'date': [],
     'region': [],
     'country': [],
     'nested': [],
     'integer': [],
     'short': [],
     'byte': [],
     'double': [],
     'float': [],
     'half_float': [],
     'scaled_float

### Copy resources from staging to production

In [8]:
try:
    resources=[]

    for dataset in stagingData['data']:

        newDataset = recreateDataset(dataset)

        resources.append({
            'type': 'dataset',
            'stagingId':dataset.get('id'),
            'productionId': newDataset['data'].get('id')
        })

        for vocabulary in dataset['attributes'].get('vocabulary'):
            newVocabulary = recreateVocabulary(newDataset['data'].get('id'), vocabulary)
            resources.append({
            'type': 'vocabulary',
            'stagingId':vocabulary.get('id'),
            'productionId': newVocabulary['data']
        })

        for layer in dataset['attributes'].get('layer'):
            newLayer = recreateLayer(newDataset['data'].get('id'), layer)
            resources.append({
            'type': 'layer',
            'stagingId':layer.get('id'),
            'productionId': newLayer['data'].get('id')
        })

        for widget in dataset['attributes'].get('widget'):
            newWidget = recreateWidget(newDataset['data'].get('id'), widget)
            resources.append({
            'type': 'widget',
            'stagingId':widget.get('id'),
            'productionId': newWidget['data'].get('id')
        })

        for metadata in dataset['attributes'].get('metadata'):
            newMetadata = recreateMetadata(newDataset['data'].get('id'), metadata)
            resources.append({
            'type': 'metadata',
            'stagingId':metadata.get('id'),
            'productionId': newMetadata['data']
        })
except:
    pass
        
    
    

{'data': {'id': '79260642-1758-46ee-a73e-297cf036fd7a', 'type': 'layer', 'attributes': {'name': 'Water Risk - Groundwater table declining trend', 'slug': 'Water-Risk-Groundwater-table-declining-trend', 'dataset': '3d010bec-30aa-4a2b-aba2-11dbf2750b55', 'application': ['aqueduct'], 'iso': [''], 'provider': 'cartodb', 'userId': '57a0aa1071e394dd32ffe137', 'default': True, 'protected': False, 'published': True, 'env': 'production', 'layerConfig': {'account': 'wri-rw', 'params_config': [], 'sql_config': [{'key': 'where', 'key_params': [{'key': 'iso', 'required': False}, {'key': 'crop', 'required': False}, {'key': 'irrigation', 'required': False}]}], 'body': {'maxzoom': 18, 'minzoom': 3, 'layers': [{'type': 'cartodb', 'options': {'sql': "with r as (SELECT basinid, value, label, aqid FROM water_risk_indicators_v3 where indicator='groundwater_decline_trend') SELECT r.aqid, value, label, the_geom_webmercator from r inner join wri_subcatchements_v3 on r.aqid=wri_subcatchements_v3.aqid", 'cartoc

In [9]:
with open(f'AQ_prod_staging_match_{datetime.now().strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
    json.dump(resources, outfile)

## Also layers ids need to be updated in the food app every time this operation happens.

## Don't forget to run the migration of AQ flood layers as they are also contained in the same app.

[AQ3-EE_layers_to_RW_API.ipynb](./AQ3-EE_layers_to_RW_API.ipynb)  
## and dataset id changed everytime this is rerun